In [1]:
import os
import sys
import unittest
import fileinput
import networkx as nx

sys.path.append(os.path.realpath(".."))

from src.datasets import Flight
from src.searchers import GraphBackTracker

In [2]:
def load_data(filename=None):
    flights = []

    for i, line in enumerate(fileinput.input(filename)):
        if i != 0:
            flights.append(process_line(line))
        else:
            start = line.rstrip()

    return start, flights


def process_line(line):
    lst = line.rstrip().split(' ')
    return Flight(lst[0], lst[1], int(lst[2]), int(lst[3]))

In [3]:
class GraphBackTracker:
    def __init__(self, start_city, flights):
        self.start_city = start_city
        self.to_visit = {flt.city_from for flt in flights}
        self.last_day = len(self.to_visit) - 1
        self.G = self._build_graph(flights)

    def _build_graph(self, flights):
        G = nx.MultiDiGraph()
        for flt in flights:
            if self._is_new_edge(flt):
                G.add_edge(flt.city_from, flt.city_to, weight=flt.price, day=flt.day)
        return G

    def _is_new_edge(self, flight):
        is_first_day_flight = flight.city_from == self.start_city and flight.day == 0
        is_last_day_flight = flight.city_to == self.start_city and flight.day == self.last_day
        is_midtrip_flight = flight.day != 0 and flight.day != self.last_day and self.start_city not in [
            flight.city_from,
            flight.city_to]
        is_same_city = flight.city_from == flight.city_to

        return (is_first_day_flight or is_last_day_flight or is_midtrip_flight) and not is_same_city

    def search(self):
        pass

In [64]:
def backtracking(G, start_city, tv):
    to_visit = list(tv)
    
    path = []
    cost = 0
    current_day = 0
    current_city = start_city
    
    i = 0
    while to_visit:
        possible_flights = [(city_from, city_to, data) for (city_from, city_to, data) in G.edges(current_city, data=True)
                           if data['day'] == current_day and city_to in to_visit]
        
        if possible_flights:
            best_flight = min(possible_flights, key=lambda (u, v, d): d['weight'])
            
            cost += best_flight[2]['weight']
            current_city = best_flight[1]
            current_day += 1
            
            to_visit.remove(current_city)
            
            path.append(Flight(best_flight[0], best_flight[1], best_flight[2]['day'], best_flight[2]['weight']))
        else:
            to_visit.append(current_city)
            last_flight = path.pop(-1)
            current_city = last_flight.city_from
            current_day -= 1
            cost -= last_flight.price
            
            for u,v,key,data in G.out_edges(last_flight.city_from,data=True,keys=True):
                if u == last_flight.city_from and v == last_flight.city_to and data['day']== last_flight.day and data['weight']==last_flight.price:
                    G.remove_edge(u,v,key=key)
                    
    return cost, path
            
            
start_city, flights = load_data('../input/3_airports_input.csv')
start_city, flights = load_data('../input/4_airports_backtrace.csv')
solver = GraphBackTracker(start_city, flights)

#print start_city
#for f in flights:
#    print f

#for e in solver.G.edges(data=True):
#    print e
    
#print solver.to_visit

print backtracking(solver.G, solver.start_city, solver.to_visit)

(400, [<Flight: PRG TXL 0 100>, <Flight: TXL BCN 1 100>, <Flight: BCN DEL 2 100>, <Flight: DEL PRG 3 100>])


In [ ]:
def search(self, dataset):

    trip = []
    #possible_flights = { day : [flight flight flight ... ] }
    # je to vpodstate stack
    possible_flights = {}

    start_city = dataset.get_starting_city()
    cities_to_visit = deepcopy(dataset.cities)
    cities_to_visit.remove(start_city)

    day = 0
    cur_city = start_city

    while(cities_to_visit):
        if day not in possible_flights.keys():
            possible_flights[day] = dataset.get_flights(cur_city,
                                                        day,
                                                        cities_to_visit=cities_to_visit,
                                                        sort_by_price=True)
        if not possible_flights[day]:
            assert day != 0, "day 0 and nowhere to go  \
                              - either a bug or no cycle in data"
            #backwards
            del possible_flights[day]

            last_flight = trip.pop(-1)
            cities_to_visit.append(cur_city)
            cur_city = last_flight.city_from
            day -= 1
            assert day >= 0, "bug: returning before day 0"

        else:
            #forward
            flight_taken = possible_flights[day].pop(0)
            trip.append(flight_taken)
            cities_to_visit.remove(flight_taken.city_to)
            cur_city = flight_taken.city_to
            day += 1

        # return to starting city:
        if not cities_to_visit:
            possibilities = dataset.get_flights(cur_city,
                                                day,
                                                cities_to_visit=[start_city],
                                                sort_by_price=True)
            if possibilities:
                trip.append(possibilities[0])
            else:
                last_flight = trip.pop(-1)
                cities_to_visit.append(cur_city)
                cur_city = last_flight.city_from
                day -= 1
                assert day >= 0, "bug: returning before day 0"

    return trip

In [ ]:
def nearest_neighbors(G, start):
    to_visit = G.nodes()
    
    cost = 0
    path = [start]
    current_day = 0
    current_city = start
    
    while to_visit:
        current_flights = [(u, v, d) for (u, v, d) in G.edges(current_city, data=True)
                           if d['day'] == current_day and v in to_visit]
        best_flight = min(current_flights, key=lambda (u, v, d): d['weight'])
        
        cost += best_flight[2]['weight']
        current_city = best_flight[1]
        current_day += 1
        
        to_visit.remove(current_city)
        path.append(current_city)
        
    return cost, path
    
cost, path = nearest_neighbors(G, start)
print path
print cost

assert len(np.unique(path)) == len(G.nodes())